Estadísticas descriptivas básicas calculadas

In [31]:
import pandas as pd
import numpy as np
from pathlib import Path

# --- PASO 1: CONFIGURACIÓN DE RUTAS Y NOMBRES ---
archivos_info = {
    'clientes': 'limpios/df_clientes_limpio.csv',
    'productos': 'limpios/df_productos_limpio.csv',
    'ventas': 'limpios/df_ventas_limpio.csv',
    'detalle_ventas': 'limpios/df_detalle_ventas_limpio.csv'
}

dataframes = {}

print("--- 1. CARGANDO DATASETS INDIVIDUALMENTE ---")

for nombre, archivo in archivos_info.items():
    try:
        # ✅ Leer como CSV desde la carpeta detectada
        ruta = find_data_dir() / archivo
        df_temp = pd.read_csv(ruta)
        dataframes[nombre] = df_temp
        print(f"✅ Cargado: {nombre} ({df_temp.shape[0]} filas, {df_temp.shape[1]} columnas)")
    except FileNotFoundError:
        print(f"❌ ERROR: Archivo '{archivo}' no encontrado.")
    except Exception as e:
        print(f"❌ ERROR al procesar '{archivo}': {e}")

--- 1. CARGANDO DATASETS INDIVIDUALMENTE ---
❌ ERROR al procesar 'limpios/df_clientes_limpio.csv': name 'find_data_dir' is not defined
❌ ERROR al procesar 'limpios/df_productos_limpio.csv': name 'find_data_dir' is not defined
❌ ERROR al procesar 'limpios/df_ventas_limpio.csv': name 'find_data_dir' is not defined
❌ ERROR al procesar 'limpios/df_detalle_ventas_limpio.csv': name 'find_data_dir' is not defined


In [ ]:
# Función para detectar la carpeta 'datos'
def find_data_dir(start: Path | None = None, names=('datos', 'data')) -> Path | None:
    if start is None:
        start = Path.cwd()
    start = Path(start).resolve()
    for parent in [start] + list(start.parents):
        for n in names:
            candidate = parent / n
            if candidate.is_dir():
                return candidate
    return None

In [32]:
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option("display.max_columns", 100)

def es_numerica(s):
    return pd.api.types.is_numeric_dtype(s)

Estadísticas descriptivas básicas calculadas

In [33]:
for nombre, df in dataframes.items():
    print(f"\n{'='*60}\n📊 ESTADÍSTICAS BÁSICAS — {nombre.upper()}\n{'='*60}")

    # Numéricas
    desc_num = df.select_dtypes(include=[np.number]).describe().T
    if not desc_num.empty:
        print("\n▶ Numéricas:")
        display(desc_num)

    # No numéricas
    desc_cat = df.select_dtypes(exclude=[np.number]).describe(include='all').T
    if not desc_cat.empty:
        print("\n▶ Categóricas / texto:")
        display(desc_cat)

Identificación del tipo de distribución

In [34]:
def clasificar_distribucion(serie):
    skew = serie.skew()
    kurt = serie.kurtosis()
    if np.isnan(skew):  # todo NaN o constante
        return "sin variación / NaN", skew, kurt
    # Heurística simple por asimetría
    if abs(skew) < 0.5:
        tipo = "aprox. normal/simétrica"
    elif skew >= 0.5:
        tipo = "sesgo a la derecha (cola derecha)"
    else:
        tipo = "sesgo a la izquierda (cola izquierda)"
    return tipo, skew, kurt

for nombre, df in dataframes.items():
    print(f"\n{'='*60}\n📈 DISTRIBUCIONES — {nombre.upper()}\n{'='*60}")

    # Numéricas: clasificación + histograma
    num_cols = df.select_dtypes(include=[np.number]).columns
    for col in num_cols:
        s = df[col].dropna()
        tipo, skew, kurt = clasificar_distribucion(s)
        print(f"- {col}: {tipo} | skew={skew:.2f}, kurtosis={kurt:.2f}")

        # Histograma + KDE
        plt.figure()
        sns.histplot(s, kde=True, bins=30)
        plt.title(f"Distribución de {col} — {nombre}")
        plt.xlabel(col)
        plt.ylabel("Frecuencia")
        plt.show()

    # Categóricas: top 10 frecuencias
    cat_cols = df.select_dtypes(exclude=[np.number]).columns
    if len(cat_cols) > 0:
        print("\n▶ Frecuencias (top 10) en categóricas:")
        for col in cat_cols:
            vc = df[col].value_counts(dropna=False).head(10)
            print(f"\n{col} (top 10):")
            display(vc.to_frame("conteo"))


Análisis de correlaciones entre variables principales

In [35]:
for nombre, df in dataframes.items():
    print(f"\n{'='*60}\n🔗 CORRELACIONES — {nombre.upper()}\n{'='*60}")
    num = df.select_dtypes(include=[np.number])
    if num.shape[1] < 2:
        print("⚠️ No hay suficientes columnas numéricas para correlación.")
        continue

    corr = num.corr(numeric_only=True)
    display(corr)

    # Mapa de calor
    plt.figure(figsize=(min(10, 1.2*len(num.columns)), min(8, 0.9*len(num.columns))))
    sns.heatmap(corr, annot=True, fmt=".2f", cmap="coolwarm", square=False)
    plt.title(f"Matriz de correlación — {nombre}")
    plt.tight_layout()
    plt.show()

    # Pares con mayor correlación (en valor absoluto), sin duplicados ni diagonal
    corr_pairs = (
        corr.where(~np.eye(corr.shape[0], dtype=bool))
            .abs()
            .stack()
            .sort_values(ascending=False)
    )
    top = corr_pairs.drop_duplicates().head(10)
    if not top.empty:
        print("\n▶ Top correlaciones (|r| más alto):")
        display(top.to_frame("abs(r)"))


Detección de outliers

In [36]:
def iqr_outliers(s):
    """Devuelve máscara booleana de outliers y límites inferior/superior según el método IQR."""
    q1 = s.quantile(0.25)
    q3 = s.quantile(0.75)
    iqr = q3 - q1
    lim_inf = q1 - 1.5 * iqr
    lim_sup = q3 + 1.5 * iqr
    mask = (s < lim_inf) | (s > lim_sup)
    return mask, lim_inf, lim_sup

for nombre, df in dataframes.items():
    print(f"\n{'='*60}\n🚨 OUTLIERS (IQR) — {nombre.upper()}\n{'='*60}")

    # ✅ Seleccionamos solo columnas numéricas que no sean IDs
    num_cols = [
        c for c in df.select_dtypes(include=[np.number]).columns
        if "id" not in c.lower()
    ]

    if len(num_cols) == 0:
        print("No hay columnas numéricas válidas (sin IDs).")
        continue

    resumen = []
    outlier_indices_global = set()

    for col in num_cols:
        s = df[col].dropna()
        if s.empty:
            continue

        mask, li, ls = iqr_outliers(s)
        idx = s.index[mask]
        resumen.append({
            "columna": col,
            "outliers": len(idx),
            "lim_inf": li,
            "lim_sup": ls
        })
        outlier_indices_global.update(idx)

        # Boxplot visual
        plt.figure()
        sns.boxplot(x=df[col])
        plt.title(f"Boxplot y outliers en {col} — {nombre}")
        plt.show()

    if resumen:
        print("\n▶ Resumen por columna:")
        display(pd.DataFrame(resumen).sort_values("outliers", ascending=False))

    if outlier_indices_global:
        print("\n▶ Filas con al menos un outlier (primeras 20):")
        display(df.loc[sorted(outlier_indices_global)].head(20))
    else:
        print("✅ Sin outliers según criterio IQR.")